In [ ]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime
import time
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, filterFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2, figures)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, IntSlider
from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

# login to myplant()
cred()
mp = MyPlant(3600)
Engine._list_cached_validations();

css = '''
.bk.panel-widget-box {
  background: #f0f0f0;
  color: black;
  /*border-radius: 5px;
  border: 1px black solid;*/
}
'''
#mp._fetch_installed_base(); # refresh local installed fleet database

In [ ]:
lookup = 'Forsa Hartmoor'
motor_num = 0
def sfun(x):
    return all([ (lookup in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T;
ddl = pd.DataFrame(fleet['serialNumber'] + ' - ' + fleet['IB Site Name'] + ' ' + fleet['Engine ID'])[0].to_list()
ddl = [m for m in ddl]


In [ ]:
import param
import panel as pn
import pandas as pd
import datetime as dt


def get_engines(query, engines):
    def sfun(x):
        return all([ (query in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
    fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
    ddl = pd.DataFrame(fleet['serialNumber'] + ' - ' + fleet['IB Site Name'] + ' ' + fleet['Engine ID'])[0].to_list()
    ddl = [m for m in ddl]
    return pn.widgets.Select(value='-', options=ddl, align='center')

query = pn.widgets.TextInput(value='Forsa Hartmoor')
engines  = pn.widgets.Select(value='-', options=ddl, align='center')
#engines = pn.bind(get_engines, query, align='center')
modes = pn.widgets.CheckBoxGroup(name='modes Group', value=['undefined','OFF','MAN','AUTO'], options=['undefined','OFF','MAN','AUTO'],inline=True, align='center')
success = pn.widgets.CheckBoxGroup(name='modes Group', value=[True,False], options=[True,False],inline=True, align='center')

# def print_engine(select_engine='-'):
#     return widget
#     widget.value = select_engine

class dFSMClass(param.Parameterized):
    select_engine           = param.ObjectSelector(objects=ddl)
#    t_from                  = param.CalendarDate(dt.date(2017, 1, 1), bounds=(dt.date(2017, 1, 1), dt.date.today()))
#    t_to                    = param.CalendarDate(dt.date.today(),  bounds=(dt.date(2017, 1, 1), dt.date.today()))

    # @param.depends('select_engine', watch=False)
    # def write_se(self):
    #     return print_engine(self.select_engine)

pn.extension(raw_css=[css])
base = dFSMClass()
pn.Row(query, engines, 'mode:', modes, 'success: ', success, css_classes=['panel-widget-box'])

In [ ]:
#eng = base.select_engine
eng = engines.value
motor_num = ddl.index(eng)
motor = fleet.iloc[motor_num]
modes = ['undefined','OFF','MAN','AUTO']
success = [True,False]
e=Engine.from_fleet(mp,motor)
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

In [ ]:


class BaseClass(param.Parameterized):
    x                       = param.Parameter(default=3.14, doc="X position")
    y                       = param.Parameter(default="Not editable", constant=True)
    string_value            = param.String(default="str", doc="A string")
    num_int                 = param.Integer(50000, bounds=(-200, 100000))
    unbounded_int           = param.Integer(23)
    float_with_hard_bounds  = param.Number(8.2, bounds=(7.5, 10))
    float_with_soft_bounds  = param.Number(0.5, bounds=(0, None), softbounds=(0,2))
    unbounded_float         = param.Number(30.01, precedence=0)
    hidden_parameter        = param.Number(2.718, precedence=-1)
    integer_range           = param.Range(default=(3, 7), bounds=(0, 10))
    float_range             = param.Range(default=(0, 1.57), bounds=(0, 3.145))
    dictionary              = param.Dict(default={"a": 2, "b": 9})


class Example(BaseClass):
    """An example Parameterized class"""
    timestamps = []

    boolean                 = param.Boolean(True, doc="A sample Boolean parameter")
    color                   = param.Color(default='#FFFFFF')
    date                    = param.Date(dt.datetime(2017, 1, 1),
                                         bounds=(dt.datetime(2017, 1, 1), dt.datetime(2017, 2, 1)))
    dataframe               = param.DataFrame(pd._testing.makeDataFrame().iloc[:3])
    select_string           = param.ObjectSelector(default="yellow", objects=["red", "yellow", "green"])
    select_fn               = param.ObjectSelector(default=list,objects=[list, set, dict])
    int_list                = param.ListSelector(default=[3, 5], objects=[1, 3, 5, 7, 9], precedence=0.5)
    single_file             = param.FileSelector(path='../../*/*.py*', precedence=0.5)
    multiple_files          = param.MultiFileSelector(path='../../*/*.py?', precedence=0.5)
    record_timestamp        = param.Action(lambda x: x.timestamps.append(dt.datetime.utcnow()), 
                                           doc="""Record timestamp.""", precedence=0.7)


In [ ]:
import panel as pn

css = '''
.bk.panel-widget-box {
  background: #f0f0f0;
  color: black;
  border-radius: 5px;
  border: 1px black solid;
}
'''

pn.extension(raw_css=[css])

pn.extension()

base = BaseClass()
pn.Row(Example.param, base.param,
css_classes=['panel-widget-box'])

In [ ]:
Example.timestamps

In [ ]:
import param
import hvplot.pandas
from bokeh.sampledata.autompg import autompg

def autompg_plot(x='mpg', y='hp', color='#058805'):
    return autompg.hvplot.scatter(x, y, c=color, padding=0.1)

columns = list(autompg.columns[:-2])


class MPGExplorer(param.Parameterized):
    x = param.Selector(objects=columns)
    y = param.Selector(default='hp', objects=columns)
    color = param.Color(default='#0f0f0f')
    
    @param.depends('x', 'y', 'color') # optional in this case
    def plot(self):
        return autompg_plot(self.x, self.y, self.color)

explorer = MPGExplorer()

pn.Row(explorer.param, explorer.plot)